In [ ]:
# Template Notebook for Engagement Classifcation Model

# ToDo/Models to run here: https://docs.google.com/document/d/18EJpFesMEl9Q7C1AZzDeq6uy7c8tDMvEJ_j58-MmiBI/edit?usp=sharing
# Use p8_data_processed.csv here: https://drive.google.com/drive/folders/19aJUAlkTMz7PcZE1q4hogFkjVtwYGcMT
# Upload code to help-seeking github: https://github.com/interaction-lab/help_seeking
# Record model results here: https://docs.google.com/spreadsheets/d/16ye54fSSEuAuDL_j56UIeDB-rIIrxq_kPbtyPRQOrVI/edit?usp=sharing

In [ ]:
# Imports

import pandas as pd
import numpy as np
from pandas import *
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# Load Data
# Warning: this will probably take some time
# Adjust file path based on your computer

file = '../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/p8_data_processed.csv'
data = pd.read_csv(file)

In [ ]:
data.head()

In [ ]:
# Drop Irrelevant Columns

# transcriptions
data = data.drop(columns=['transcript_spk_0', 'transcript_spk_1', 'transcript_spk_2'])

# raw ros messages
data = data.drop(columns=['ros_PARTICIPANT_STATE', 'ros_ROBOT_STATE'])

# more for counting purposes
data = data.drop(columns=['game_start', 'game_correct', 'game_incorrect', 'mistake_made'])

data = data.drop(columns=['participant', 'session_date'])

# use one-hot encoded difficulty and skill, don't use activity for now
data = data.drop(columns=['difficulty', 'skill', 'activity'])

In [ ]:
# What Non Open-Face/Open-Pose Columns? 
# Feature Dictionary Documentation here: https://docs.google.com/document/d/1RSygoLwsM1PKEIOoqDOaEo3lRBZBLZJuzbMSY5K_6zA/edit?usp=sharing

only_ofop = []
performance_features = []
for i in data.columns:
    # open face/open pose columns begin with of_ or op_
    if 'op_' not in i and 'of_' not in i:
        print(i)
    else:
        only_ofop.append(i)

In [ ]:
# Label Analysis

print(data['engagement'].value_counts())
print()

print(1,data['engagement'].value_counts()[1]/sum(data['engagement'].value_counts()))
print(0,data['engagement'].value_counts()[0]/sum(data['engagement'].value_counts()))

### Decision 1: What features to use?

In [ ]:
# TODO: Drop Some OpenFace/OpenPose Columns 

# Run a model with ONLY open face/ open pose columns
only_ofop.append('timestamp')
only_ofop.append('session_num')
only_ofop.append('engagement')
#data = data[only_ofop]

# Run a model with only critical open face/open pose columns as determined by Bala's feature analysis

# Run a model with only hand-picked features
hand_picked = ['confidence', 'success', 'gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z', 'gaze_angle_x', 'gaze_angle_y']
for i,c in enumerate(hand_picked):
    hand_picked[i] = 'of_' + c
hand_picked.append('op_Number of People')

for i in performance_features:
    hand_picked.append(i)

#data = data[hand_picked]

### Decision 2: Train Test Split?

In [ ]:
# (Optional) Sort Data Chronologically
# Important for Sequential Models! 

# data = data.sort_values(['session_num', 'timestamp'], ascending=[True, True])

In [ ]:
# Make sure engagement is first column

cols = list(data)
cols.insert(0, cols.pop(cols.index('engagement')))
data = data[cols]

In [ ]:
# Train Test Split 2: Train on Earlier Sessions, Test on Later Sessions

session_threshold = 8 # train on sessions <= 8, test on sessions > 8

train = data[(data['session_num'] <= 8.0)] 
test = data[(data['session_num'] > 8)]

# drop columns needed for split
train = train.drop(columns=['session_num', 'timestamp'])
test = test.drop(columns=['session_num', 'timestamp'])

X_train2 = train.iloc[:,1:]
y_train2 = train.iloc[:, 0]

X_test2 = test.iloc[:, 1:]
y_test2 = test.iloc[:, 0]

In [ ]:
# Train Test Split 1: Random 70-30 Split

# don't need these columns anymore
data = data.drop(columns=['session_num', 'timestamp'])

# Note: random_state: make sure we get same split every time 
X_train1, X_test1, y_train1, y_test1 = train_test_split(data.iloc[:,1:], data.iloc[:,0], test_size=0.3, random_state=42)

### Decision 3: Implement Your Model!